In [1]:
import phoenix
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

phoenix.launch_app(host="localhost", port=6006)
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=OTLPSpanExporter(endpoint="http://localhost:6006/v1/traces")))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

c:\Users\joanp\Projects\ctgov_rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.
❗️ The launch_app `host` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_HOST` environment variable instead.
🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [2]:
def fromToCtTo_query(from_node:str, from_property:str, to_node:str, to_property:str)->str:
    query = """
    WITH node, score
    OPTIONAL MATCH path = (node)-[:{from_node}ToStudyAssociation]->(ct:ClinicalTrial)-[:StudyTo{to_node}Association]->(target:{to_node})
    WITH node.{from_property} AS from_node_txt, COLLECT(DISTINCT target.{to_property}) AS to_node_list, max(score) AS score // deduplicate parents
    RETURN "{from_node}: "+from_node_txt+". {to_node}: "+apoc.text.join(to_node_list, ', ') AS text, score, {{}} AS metadata
    """
    cmd = query.format(from_node=from_node, from_property=from_property, to_node=to_node, to_property=to_property)
    return cmd

In [3]:
def fromToCt_query(from_node:str, from_property:str, ct_properties:list[str])->str:
    
    ct_properties_str = ", ".join([f"{p} = \"+ct.{p}+\" " for p in ct_properties])+"\""
    
    query = """
    WITH node, score
    OPTIONAL MATCH (node)-[:{from_node}ToStudyAssociation]->(ct:ClinicalTrial)
    WITH node, ct, max(score) AS score // deduplicate parents
    RETURN "{from_node}: "+node.{from_property}+". ClinicalTrial: {ct_properties_str} AS text, score, {{}} AS metadata
    """
    cmd = query.format(from_node=from_node, from_property=from_property, ct_properties_str=ct_properties_str)
    return cmd

In [4]:
from  sentence_transformers import SentenceTransformer
biobert = SentenceTransformer("dmis-lab/biobert-base-cased-v1.1")

In [ ]:
# from dotenv import  load_dotenv
# load_dotenv("../.env")

In [5]:
import os
import dspy
from dspy.retrieve.neo4j_rm import Neo4jRM

os.environ["NEO4J_URI"] = "bolt://localhost:7689"
os.environ["NEO4J_USERNAME"] = "tester"
os.environ["NEO4J_PASSWORD"] = "password"
os.environ["NEO4J_DATABASE"] = "ctgov"

#lm = dspy.OpenAI(model='gpt-3.5-turbo')
lm = dspy.OllamaLocal(model="mistral",stop=["[INST]", "[/INST]"], max_tokens=500, timeout_s=2000)
dspy.settings.configure(lm=lm)



neo4j_retriever = Neo4jRM(
    index_name="intervention_biobert_emb",
    text_node_property="name",
    k=10,
    embedding_provider="huggingface",
    embedding_model="dmis-lab/biobert-base-cased-v1.1",
    retrieval_query = fromToCtTo_query("Intervention", "name", "AdverseEvent", "term"),
)

neo4j_retriever.embedder = biobert.encode

# dspy.settings.configure(rm=neo4j_retriever)

In [9]:
cond2ct_retriever = Neo4jRM(
    index_name="condition_biobert_emb",
    text_node_property="name",
    k=10,
    embedding_provider="huggingface",
    embedding_model="dmis-lab/biobert-base-cased-v1.1",
    retrieval_query = fromToCt_query("Intervention", "name", ["id", "study_type","brief_title"]),
)

cond2ct_retriever.embedder = biobert.encode

In [11]:
cond2ct_retriever(["Burkitt's Lymphoma"], k=3)

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.05it/s]

[{'long_text': None}, {'long_text': None}, {'long_text': None}]

In [6]:
class InterventionToAdverseEvent(dspy.Module):
    name = "InterventionToAdverseEvent"
    input_variable = "intervention"
    desc = "Get the Adverse Events associated to a medical Intervention tested in a clinical trial"
    
    def __init__(self):
        self.retriever = Neo4jRM(index_name="intervention_biobert_emb", 
                                 text_node_property="name",
                                embedding_provider="huggingface",
                                embedding_model="dmis-lab/biobert-base-cased-v1.1",
                                 k=10, 
                                 retrieval_query=fromToCtTo_query("Intervention", "name", "AdverseEvent", "term"))
        self.retriever.embedder = biobert.encode
        
    def __call__(self, intervention:str, k:int=5)->str:
        print(f"InterventionToAdverseEvent({intervention})")
        response = self.retriever(intervention, k)
        response = "\n".join([x["long_text"] for x in response])
        print(response)
        return response

In [7]:
class InterventionToCt(dspy.Module):
    name = "InterventionToCt"
    input_variable = "intervention"
    desc = "Get the Clinical Trials associated to a medical Intervention"
    
    def __init__(self):
        self.retriever = Neo4jRM(index_name="intervention_biobert_emb", 
                                 text_node_property="name",
                                embedding_provider="huggingface",
                                embedding_model="dmis-lab/biobert-base-cased-v1.1",
                                 k=10, 
                                 retrieval_query=fromToCt_query("Intervention", "name", ["id", "study_type","brief_title"])
                                 )
        self.retriever.embedder = biobert.encode
        
    def __call__(self, intervention:str, k:int=5)->list:
        print(f"InterventionToCt({intervention})")
        response = self.retriever(intervention, k)
        response = "\n".join([x["long_text"] for x in response])
        print(response)
        return response

In [8]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField()


# Pass signature to ReAct module
tools = [InterventionToAdverseEvent(), InterventionToCt()]
react_module = dspy.ReAct(BasicQA, tools=tools, max_iters=3)


# Call the ReAct module on a particular input
question = 'What are the adverse events associated with drug Tralokinumab'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

InterventionToAdverseEvent(Tralokinumab)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.69s/it]


Intervention: Tralokinumab. AdverseEvent: Hypertension, Dyspnoea, Cough, Headache, Back pain, Arthralgia, Viral upper respiratory tract infection, Sinusitis, Oral candidiasis, Injection site pruritus, Injection site pain, Injection site erythema, Fatigue, Asthma, Vaginal prolapse, Acute kidney injury, Uterine leiomyoma, Breast cancer female, Tendonitis, Rotator cuff syndrome, Weight decreased, Pulmonary function test abnormal, Rib fracture, Hand fracture, Urinary tract infection, Influenza, Bronchitis, Colitis, Epistaxis, Rhinorrhoea, Oropharyngeal pain, Procedural pain, Weight increased, Non-cardiac chest pain, Injection site swelling, Rash, Nausea, Vomiting, Upper respiratory tract infection, Angioedema, Pulmonary embolism, Pharyngeal oedema, Atelectasis, Prostatitis, Metrorrhagia, Hallucination, Depression, Vertebrobasilar insufficiency, Vascular encephalopathy, Ischaemic stroke, Haemorrhagic stroke, Dizziness, Cerebral haemorrhage, Carpal tunnel syndrome, Uterine cancer, Cervix car

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.47it/s]


Intervention: Tralokinumab. AdverseEvent: Hypertension, Dyspnoea, Cough, Headache, Back pain, Arthralgia, Viral upper respiratory tract infection, Sinusitis, Oral candidiasis, Injection site pruritus, Injection site pain, Injection site erythema, Fatigue, Asthma, Vaginal prolapse, Acute kidney injury, Uterine leiomyoma, Breast cancer female, Tendonitis, Rotator cuff syndrome, Weight decreased, Pulmonary function test abnormal, Rib fracture, Hand fracture, Urinary tract infection, Influenza, Bronchitis, Colitis, Epistaxis, Rhinorrhoea, Oropharyngeal pain, Procedural pain, Weight increased, Non-cardiac chest pain, Injection site swelling, Rash, Nausea, Vomiting, Upper respiratory tract infection, Angioedema, Pulmonary embolism, Pharyngeal oedema, Atelectasis, Prostatitis, Metrorrhagia, Hallucination, Depression, Vertebrobasilar insufficiency, Vascular encephalopathy, Ischaemic stroke, Haemorrhagic stroke, Dizziness, Cerebral haemorrhage, Carpal tunnel syndrome, Uterine cancer, Cervix car

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
lm.inspect_history(n=10)

ReAct

In [ ]:
# source https://discord.com/channels/1161519468141355160/1213167378955767859/1213168250414702642

import dspy
import datetime
import requests

# lm = dspy.OpenAI(model='gpt-3.5-turbo')
lm = dspy.OllamaLocal(model="mistral",stop=["[INST]", "[/INST]"], max_tokens=500)
dspy.settings.configure(lm=lm)

class GiveTime(dspy.Module):
    name  = "GiveTime"
    input_variable = "empty"
    desc = "takes an empty string and returns the current local time"

    def __init__(self):
        pass

    def __call__(self, *args, **kwargs):
        return datetime.datetime.now().strftime("%H:%M:%S")
    

class GetCoinPrice(dspy.Module):
    name = "GetCoinPrice"
    input_variable = "symbol"
    desc = "takes a coin ticker and returns the current price of the coin"

    def __init__(self):
        pass

    def __call__(self, *args, **kwargs):
        ticker = args[0]
        coin_prices = requests.get("https://api.coincap.io/v2/assets").json()["data"]
        for c in coin_prices:
            if c["symbol"] == ticker:
                price = c["priceUsd"]
                # Format the price to 2 decimal places and add a dollar sign
                pretty_price = f"${float(price):.2f}"
                return pretty_price
        return "Coin not found"

# Define a simple signature for basic question answering

class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


# Pass signature to ReAct module
tools = [GiveTime(), GetCoinPrice()]
react_module = dspy.ReAct(BasicQA, tools=tools)


# Call the ReAct module on a particular input
question = 'What is the time right now????'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

In [ ]:
# Call the ReAct module on a particular input
question = 'What is dogecoin price right now?'
result = react_module(question=question)

print(f"Question: {question}")
print(f"Final Predicted Answer (after ReAct process): {result.answer}")

txt2SQL

In [ ]:
import os

os.environ["DPS_CACHEBOOL"] = "False"
import json

import dspy
from dotenv import load_dotenv

In [ ]:
from langchain_community.utilities import SQLDatabase

In [ ]:
load_dotenv(".env")
AACT_USER = os.getenv("AACT_USER")
AACT_PWD = os.getenv("AACT_PWD")

In [ ]:
tables = [
    "browse_interventions",
    "sponsors",
    "outcome_analysis_groups",
    "detailed_descriptions",
    "facilities",
    "studies",
    "outcomes",
    "browse_conditions",
    "outcome_analyses",
    "keywords",
    "eligibilities",
    "id_information",
    "design_group_interventions",
    "reported_events",
    "brief_summaries",
    "designs",
    "drop_withdrawals",
    "outcome_measurements",
    "countries",
]

In [ ]:
database = "aact"
host = "aact-db.ctti-clinicaltrials.org"
user = AACT_USER
password = AACT_PWD
port = 5432
db_uri = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
sql_db = SQLDatabase.from_uri(db_uri, include_tables=tables)

In [ ]:
with open("./src/txt_2_sql/aact_schema.txt", "r") as f:
    aact_schema =  f.readlines()

aact_schema = "".join(aact_schema).replace("\n\n", "\n")

In [ ]:
with open('./src/txt_2_sql/common_sql_mistakes.txt', "r") as f:
    common_mistakes = f.readlines()

In [ ]:
common_mistakes = "".join(common_mistakes)

In [ ]:
import yaml

with open('./src/txt_2_sql/aact_schema.yaml', 'r') as f:
    aact_schema = yaml.safe_load(f)


In [ ]:
class Text2Sql(dspy.Signature):
    """Given an input question and a SQL db schema, create a syntactically correct PostgreSQL query to run. 
    Never query for all the columns from a specific table, only ask for a few relevant columns given the question.
    Pay attention to use only the column names that you can see in the schema description. 
    Be careful to not query for columns that do not exist. Pay attention to which column is in which table.
    Also, qualify column names with the table name when needed
    """

    context:str = dspy.InputField(prefix="Schema:", desc="SQL db schema")
    question:str = dspy.InputField(prefix="Question:", desc="user question")
    sql_query:str = dspy.OutputField(prefix="SQLQuery:", desc="SQL query that answers user question")

In [ ]:
class CheckSqlQuery(dspy.Signature):
    "Take a SQL query and a list of common mistakes and suggest a revised SQL query"
    context:str = dspy.InputField(prefix="Common mistakes:", desc="Common SQL syntax mistakes")
    sql_query:str = dspy.InputField(prefix="SQLQuery:")
    revised_sql:str = dspy.OutputField(prefix="Revised SQLQuery:",)

In [ ]:
class CheckSqlSchema(dspy.Signature):
    "Take a SQL query and a SQL db schema and suggest a revised SQL query"
    context:str = dspy.InputField(prefix="SQL db schema:")
    sql_query:str = dspy.InputField(prefix="SQLQuery:")
    revised_sql:str = dspy.OutputField(prefix="Revised SQLQuery:")

In [ ]:
class CheckSqlError(dspy.Signature):
    "Take a SQL query and the error produced when running it and suggests a revised SQL query"
    db_schema = dspy.InputField(desc="SQL DB schema", prefix="Schema: ")
    question = dspy.InputField(desc="User question", prefix="Question: ")
    sql_query = dspy.InputField(desc="Original SQL query", prefix="SQLQuery: ")
    error = dspy.InputField(
        desc="Exception through when running SQL query", prefix="Exception: "
    )
    revised_sql = dspy.OutputField(desc="Revised SQL query", prefix="Revised:")

In [ ]:
class QuestionSqlAnswer(dspy.Signature):
    question:str = dspy.InputField(prefix="Question: ")
    sql_output:str = dspy.InputField(prefix="SQL output: ")
    answer:str = dspy.OutputField(prefix="Answer:")

In [ ]:
from typing import Literal

In [ ]:
class PickListTables(dspy.Signature):
    "Based on a user question and SQL db schema. Write a list of tables that would be relevant"
    context:dict = dspy.InputField(prefix="Schema: ")
    question:str = dspy.InputField(prefix="Question: ")
    answer:list[Literal[*(x for x in aact_schema.keys())]] = dspy.OutputField()

In [ ]:
from typing import List
from pydantic import BaseModel

class StringList(BaseModel):
    strings: List[str]

# Example usage:
input_data = {"strings": ["apple", "banana", "cherry"]}
string_list = StringList(**input_data)
print(string_list)

In [ ]:
class SqlTablesList(BaseModel):
    sql_tables: List[str]

In [ ]:
acct_tables = SqlTablesList(sql_tables=list(aact_schema.keys()))

In [ ]:
def write_schema_txt(schema:dict)->str:
    out_str= ""
    for table in schema.keys():
        out_str+= f"table name: {table}\n"
        out_str+= f"table description: {schema[table]['description']}\n"
        out_str+= f"table schema: {schema[table]['schema']}\n"
        out_str+= f"{schema[table]['example']}\n"
    return out_str

In [ ]:
aact_schema_str = write_schema_txt(aact_schema)
aact_schema_str = aact_schema_str.replace("\n\n","\n")

In [ ]:
class Txt2SqlAgent(dspy.Module):
    def __init__(self, sql_schema:str, common_mistakes=str) -> None:
        super().__init__()
        self.text_2_sql = dspy.Predict(Text2Sql)
        self.review_query = dspy.Predict(CheckSqlQuery)
        self.review_schema = dspy.Predict(CheckSqlSchema)
        self.sql_schema = sql_schema
        self.common_mistakes = common_mistakes
    
    def forward(self, question:str) ->str:
        response = {}
        response["txt2sql"] = self.text_2_sql(context=self.sql_schema, question=question)
        response["check_sql_query"] = self.review_query(context=self.common_mistakes, sql_query=response["txt2sql"]["sql_query"] )
        response["check_sql_schema"] =self.review_schema(context=self.sql_schema, sql_query=response["check_sql_query"]["revised_sql"] )
        return response
        

In [ ]:
class ChainOfTables(dspy.Module):
    def __init__(self, sql_db: SQLDatabase) -> None:
        super().__init__()
        self.sql_db = sql_db
        self.text_2_sql = dspy.Predict(Text2Sql)
        self.review_query = dspy.Predict(CheckSqlQuery)
        self.question_sql_answer = dspy.Predict(CheckSqlSchema)
        self.schema = sql_db.get_table_info(sql_db.get_usable_table_names())

    def forward(self, question: str, n: int = 3):
        attempts = 0
        sql_output = None
        db_schema = self.schema
        response = self.text_2_sql(question=question, db_schema=db_schema)
        sql_query = response.sql_query
        # return response
        while attempts < n and sql_output is None:
            try:
                sql_output = self.sql_db.run(sql_query)
            except Exception as e:
                response = self.review_query(
                    question=question,
                    db_schema=db_schema,
                    sql_query=sql_query,
                    error=str(e),
                )
                sql_query = response.revised_sql
                print(sql_query)
            attempts += 1
        if sql_output is None:
            return {
                "question": question,
                "sql_query": sql_query,
                "answer": "Sorry I could not reply your question",
            }
        return self.question_sql_answer(question=question, sql_output=sql_output)

In [ ]:
#lm = dspy.OllamaLocal(model="mistral", stop=["\n", "\n\n"])
# lm = dspy.OllamaLocal(model="mistral",stop=["[INST]", "[/INST]"])
# lm = dspy.OllamaLocal(model="mistral")
lm = dspy.OllamaLocal(model="mistral",stop=["[INST]", "[/INST]"], max_tokens=500)
dspy.settings.configure(lm=lm, temperature=0.0 )

In [ ]:
aact_schema_str.replace("\n\n","\n")

In [ ]:
#aact_sql_rag = ChainOfTables(sql_db)
acct_sql = Txt2SqlAgent(aact_schema_str, common_mistakes)

question = (
    "Which clinical trial ids are associated with the condition 'Asthma' and conducted in"
    "the United States, China and India, while involving the"
    "intervention 'Xhance', and reporting more than five affected subjects"
    "in either deaths or serious adverse events?"
)

question = question.replace("\n", " ")
response = acct_sql(question=question)

In [ ]:
acct_summary = [{"name": table, "description" : aact_schema[table]['description']} for table in aact_schema.keys()]

In [ ]:
import numpy as np

In [ ]:
acct_summary = np.array2string(np.array(acct_summary))

In [ ]:
table_picker = dspy.Predict(PickListTables)
response = table_picker(context=acct_summary, question="Get the clinical study title")

In [ ]:
response

In [ ]:
print(response["txt2sql"]["rationale"])

In [ ]:
print(response["check_sql_query"]["rationale"])

In [ ]:
print(response["txt2sql"]["sql_query"])

In [ ]:
print(response["check_sql_query"]["revised_sql"])

In [ ]:
print(response["check_sql_schema"]["revised_sql"])

In [ ]:
lm = dspy.OllamaLocal(model="mistral",stop=["[INST]", "[/INST]"], max_tokens=2000)
dspy.settings.configure(lm=lm, temperature=0.7)

qa = dspy.Predict("question -> long_answer")
response = qa(question="Write a long text about Health Data Science")

In [ ]:
print(response["long_answer"])